In [74]:
# textsumExample to get a feel for text summerization
# importing libraries for text summerization
import io
import json
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import networkx as nx
import math
from sklearn.metrics.pairwise import cosine_similarity
from gensim.summarization.summarizer import summarize

In [75]:
# function to remove these stopwords from our dataset
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

# Extract word vectors
# using pre-trained Wikipedia 2014 + Gigaword 5 GloVe vectors
# you can download it @ https://nlp.stanford.edu/data/glove.6B.zip

def get_rank(myList = [],*args):
    textrank = ''
    for i in range(math.ceil(len(ranked_sentences)/3)):
        textrank += (ranked_sentences[i][1])
    return textrank    

word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

# create dataframe for article to munipulate
df = pd.read_json("update.json")
with open('update.json',encoding="utf8") as data_file:
    data = json.load(data_file)

#normalize df
df_fields = pd.json_normalize(data,max_level=1)


In [76]:
#sumtext = df_fields['fields.description'][0] 
#df.fields[0]['description']


#sumtext_s = (summarize(sumtext,ratio=0.15))

In [77]:
sumtext = df_fields['fields.description'][0] 

In [78]:
sentences = []
#for s in df_fields['fields.description']:
sentences.append(sent_tokenize(df_fields['fields.description'][0]))

sentences = [y for x in sentences for y in x] # flatten list

In [79]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [80]:
stop_words = stopwords.words('english')

In [81]:
# create vectors for our sentences
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [82]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [83]:
# initialize the matrix with cosine similarity scores.
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [84]:

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [85]:
#Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [86]:
for i in range(5):
    print(ranked_sentences[i][1])

Ensure that projects are completed on time and they meet all business, systems, and quality requirements.
Manages communication plan for project status, issues, and risks; provides project performance metrics with a focus on status, budgets, and projections.
Manages activities of project teams, controlling the indirect management of resources and schedules to ensure timely completion.
Position Type:
Full Time

Responsibilities:

	Define, orchestrate, and manage the holistic project life cycle, delivery cadence, and release plan.
Collaborate with IT service teams to develop detailed work plans, including work breakdown structures, project milestones, risk assessment/management plans, staffing needs, project timelines, and budgets.


In [87]:
text = get_rank(ranked_sentences)

In [88]:
text

'Ensure that projects are completed on time and they meet all business, systems, and quality requirements.Manages communication plan for project status, issues, and risks; provides project performance metrics with a focus on status, budgets, and projections.Manages activities of project teams, controlling the indirect management of resources and schedules to ensure timely completion.Position Type:\nFull Time\n\nResponsibilities:\n\n\tDefine, orchestrate, and manage the holistic project life cycle, delivery cadence, and release plan.Collaborate with IT service teams to develop detailed work plans, including work breakdown structures, project milestones, risk assessment/management plans, staffing needs, project timelines, and budgets.This includes acquiring resources and coordinating the efforts of team members to deliver projects according to plan.Minimum of six (6) years directly related experience in an IT project management capacity, including all aspects of process development and e

In [89]:
t = "Promote our values and mission, and serve as a role model for kids of all\n backgrounds and ages.Instructors can set their availability and generally work with their students at a ,regular time once or twice a week.All classes are taught remotely, using the curriculum and digital tools developed by Juni.We've designed the instructor role to be a flexible part-time position.Commitment to providing students a great educationInstructors\n will use the curriculum and ,teaching tools developed by Juni Learning.We look for instructors with a great attitude and desire to help kids build resilience and grit alongside their coding skills."

In [92]:
print(summarize(t))


'Ensure that projects are completed on time and they meet all business, systems, and quality requirements.Manages communication plan for project status, issues, and risks; provides project performance metrics with a focus on status, budgets, and projections.Manages activities of project teams, controlling the indirect management of resources and schedules to ensure timely completion.Position Type:\nFull Time\n\nResponsibilities:\n\n\tDefine, orchestrate, and manage the holistic project life cycle, delivery cadence, and release plan.Collaborate with IT service teams to develop detailed work plans, including work breakdown structures, project milestones, risk assessment/management plans, staffing needs, project timelines, and budgets.This includes acquiring resources and coordinating the efforts of team members to deliver projects according to plan.Minimum of six (6) years directly related experience in an IT project management capacity, including all aspects of process development and e

In [91]:
df.fields[0]['description'] = text

In [97]:
t = summarize(text)

In [98]:
if (t == ''):
    df.fields[0]['description'] = text
else:
    df.fields[0]['description'] = t

In [99]:
df.fields[0]['description']

"Promote our values and mission, and serve as a role model for kids of all backgrounds and ages.Instructors can set their availability and generally work with their students at a regular time, once or twice a week.All classes are taught remotely, using the curriculum and digital tools developed by Juni.We've designed the instructor role to be a flexible part-time position.Commitment to providing students a great education:\n\n\n\nInstructors will use the curriculum and teaching tools developed by Juni Learning.We look for instructors with a great attitude and desire to help kids build resilience and grit alongside their coding skills."

In [96]:
for z in range(len(df)):
    sumtext = df_fields['fields.description'][z]

    sentences = []
    #for s in df_fields['fields.description']:
    sentences.append(sent_tokenize(df_fields['fields.description'][z]))

    sentences = [y for x in sentences for y in x] # flatten list

    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]

    stop_words = stopwords.words('english')

    # create vectors for our sentences
    sentence_vectors = []
    for i in clean_sentences:
      if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((100,))
      sentence_vectors.append(v)

      # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])

    # initialize the matrix with cosine similarity scores.
    for i in range(len(sentences)):
      for j in range(len(sentences)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)

    #Summary Extraction
    ranked_sentences = ''
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    text = get_rank(ranked_sentences)
    t = summarize(text)
    if (t == ''):
        df.fields[z]['description'] = text
    else:
        df.fields[z]['description'] = t
    df.fields[z]['description'] = text


In [100]:
x={}
for i in range(len(df)):
    temp={}
    temp["model"] = "jobs.jobsinfo"
    temp["pk"]= i
    x[i]=temp
    x[i]["fields"]={
        "JobTitle":(df.fields[i]['JobTitle']),
        "organization":(df.fields[i]['organization']),
        "description": df.fields[i]['description'],
        "salary":(df.fields[i]['salary']),
        "place":df.fields[i]['place'],
        "link":df.fields[i]['link'],
        "site":df.fields[i]['site']}
x=list(x.values())
    
with io.open('update_summary.json', 'w',encoding='windows-1252') as fp:
    json.dump(x,fp)
#Note: My django did not like indents so I got rid of them

''